In [ ]:
import h5py
import numpy as np

def load_stream_evolution(filename):
    with h5py.File(filename, 'r') as file:
        # Cluster trajectory (single point mass)
        cluster_positions = file['nbody/pos'][:]
        cluster_velocities = file['nbody/vel'][:]
        cluster_times = file['nbody/time'][:]

        # Stream particle trajectories
        stream_positions = file['stream/pos'][:]
        stream_velocities = file['stream/vel'][:]
        stream_times = file['stream/time'][:]

    return {
        'cluster': {
            'pos': cluster_positions,
            'vel': cluster_velocities,
            'time': cluster_times
        },
        'stream': {
            'pos': stream_positions,
            'vel': stream_velocities,
            'time': stream_times
        }
    }

streams = load_stream_evolution("ngc6569_stream_evolution.h5")


In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches

def load_stream_evolution(filename):
    """Load cluster and stream evolution data from HDF5 file"""
    with h5py.File(filename, 'r') as file:
        cluster_positions = file['nbody/pos'][:]  # (3, n_times, 1)
        stream_positions = file['stream/pos'][:]   # (3, n_times, n_particles)
        times = file['nbody/time'][:]

    return {
        'cluster_pos': cluster_positions,
        'stream_pos': stream_positions,
        'times': times
    }

def create_stream_animation(filename, output_gif):
    """Create animated gif of tidal stream evolution"""

    # Load the data
    data = load_stream_evolution(filename)
    cluster_positions = data['cluster_pos']
    stream_positions = data['stream_pos']
    times = data['times']

    n_timesteps = len(times)

    # Set up the 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set consistent axis limits
    all_positions = np.concatenate([
        cluster_positions.reshape(3, -1),
        stream_positions.reshape(3, -1)
    ], axis=1)

    # Remove NaN values for limit calculation
    valid_positions = all_positions[:, ~np.isnan(all_positions).any(axis=0)]

    axis_margin = 1  # kpc
    x_lims = [valid_positions[0].min() - axis_margin, valid_positions[0].max() + axis_margin]
    y_lims = [valid_positions[1].min() - axis_margin, valid_positions[1].max() + axis_margin]
    z_lims = [valid_positions[2].min() - axis_margin, valid_positions[2].max() + axis_margin]

    def animate_timestep(frame):
        """Update plot for each timestep"""
        ax.clear()

        # Current cluster position (single point)
        cluster_x = cluster_positions[0, frame, 0]
        cluster_y = cluster_positions[1, frame, 0]
        cluster_z = cluster_positions[2, frame, 0]

        # Current stream positions (many particles)
        stream_x = stream_positions[0, frame, :]
        stream_y = stream_positions[1, frame, :]
        stream_z = stream_positions[2, frame, :]

        # Remove NaN particles (not yet stripped)
        valid_stream = ~np.isnan(stream_x)

        # Plot stream particles
        if np.any(valid_stream):
            ax.scatter(stream_x[valid_stream],
                      stream_y[valid_stream],
                      stream_z[valid_stream],
                      s=1, c='blue', alpha=0.6, label='Stream')

        # Plot cluster
        ax.scatter([cluster_x], [cluster_y], [cluster_z],
                  s=100, c='red', marker='*', label='NGC 6569')

        # Set consistent view
        ax.set_xlim(x_lims)
        ax.set_ylim(y_lims)
        ax.set_zlim(z_lims)
        ax.set_xlabel('X [kpc]')
        ax.set_ylabel('Y [kpc]')
        ax.set_zlabel('Z [kpc]')

        # Time info
        current_time = times[frame]
        ax.set_title(f'NGC 6569 Tidal Stream Evolution\nTime: {current_time:.1f} Myr')
        ax.legend()

        return ax,

    # Create animation
    animation = FuncAnimation(fig, animate_timestep, frames=n_timesteps,
                            interval=100, blit=False, repeat=True)

    # Save as gif
    print(f"Saving animation to {output_gif}...")
    animation.save(output_gif, writer='pillow', fps=60)
    print("Animation saved!")

    return animation


create_stream_animation("ngc6569_stream_evolution.h5",'ngc6569_evolution_high_framerate.gif')